In [1]:
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sns 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier  , AdaBoostClassifier , GradientBoostingClassifier
from sklearn.metrics import precision_score , recall_score 
import xgboost as xgb 
import pandas as pd 
import sklearn.neighbors._base
from os import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest 
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
sns.set_style('whitegrid')
%load_ext autoreload
%autoreload 2
sns.set_style('whitegrid')
from IPython.display import display

In [2]:
def details(data_sent , comments = ''):
    if(comments):
        print(comments)
    sp = (data_sent.isna().sum().sum())/ data_sent.size
    print('________________________________________________')
    print('Sparsity in the data : {:.2f}'.format(sp))
    num_rows = data_sent.shape
    print('Data shape' , num_rows)
    #display(data_sent['class'].value_counts())
    print('Number of sources : ')
    cl = data_sent['class'].unique()
    for c in cl:
        num_src = len(data_sent[data_sent['class']==c]['src_id'].unique())
        num_obs = len(data_sent[data_sent['class']==c])
        print(c ,' \t ' , num_src , '\t' , num_obs)

In [3]:
from features import phot_flux , en_flux , hard, powlaw_fit , bb_fit , brems_fit , intra_obs_var , inter_ob_var , info_col , phot_flux_hilim , phot_flux_lolim , en_flux_hilim , en_flux_lolim

In [4]:
info_col_cl =  info_col+['class']

In [5]:
data_imp_v2 = pd.read_csv('imputed_data/old-data-src-imp-all-imp.csv' , index_col='obs_id')

In [6]:
details(data_imp_v2)

________________________________________________
Sparsity in the data : 0.00
Data shape (1812, 58)
Number of sources : 
CV  	  60 	 994
LX  	  58 	 521
PL  	  92 	 297


In [7]:

df_comb = pd.DataFrame()
for s in data_imp_v2['src_id'].unique()[:]:
    temp = data_imp_v2[data_imp_v2['src_id']==s]
    temp_id =  temp[info_col_cl]
    temp_val = temp.drop(columns=info_col_cl)
    temp_mean = temp_val.mean().to_frame().T
    temp_id_top = temp_id.iloc[0:1].reset_index()
    temp_final = pd.concat([temp_id_top , temp_mean] , axis=1).set_index('obs_id')
    df_comb = df_comb.append(temp_final)
display(df_comb)


,src_id,num_obs,src_n,name,ra,dec,livetime,significance,likelihood,class,...,var_prob_b,photflux_aper_s,var_inter_index_h,var_inter_index_m,flux_aper_lolim_m,photflux_aper_lolim_m,flux_aper_s,photflux_aper_lolim_s,flux_aper_lolim_s,var_inter_index_s
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,33,[HPH2013]_176__________,2CXO J004245.0+411520,10.68773,41.25582,4903.5,11.31,304.46460,CV,...,0.487219,6.086708,5.191818,5.239091,15.576974,7.001916,14.923809,6.748327,15.448448,4.211515
CV_1_obs_0,CV_1,104,[HPH2013]_142__________,2CXO J004242.9+411543,10.67917,41.26198,4035.0,77.08,27912.78000,CV,...,0.212598,4.817967,6.000000,5.000000,13.739299,5.048415,13.651575,5.008563,13.836365,5.000000
CV_10_obs_0,CV_10,33,[HPH2013]_123__________,2CXO J004241.1+411544,10.67166,41.26248,4903.5,23.44,1609.49500,CV,...,0.470838,6.000597,5.258182,5.257576,15.485675,6.897611,14.818668,6.649405,15.381830,4.182727
CV_11_obs_0,CV_11,33,[HPH2013]_100__________,2CXO J004237.0+411428,10.65458,41.24113,4903.5,37.03,4392.42200,CV,...,0.488016,6.035474,5.307879,5.270909,15.559237,6.967713,14.802322,6.711295,15.410041,4.202727
CV_12_obs_0,CV_12,33,[HPH2013]_108__________,2CXO J004239.0+411325,10.66268,41.22376,4903.5,14.30,928.95920,CV,...,0.478749,5.987794,5.211250,5.239062,15.492817,6.881683,14.787455,6.657263,15.368013,4.190000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PL_94_obs_0,PL_94,1,PSR_J1826-1256,2CXO J182608.5-125634,276.53560,-12.94294,73884.4,12.96,554.72710,PL,...,0.111000,6.576754,5.650000,5.140000,14.659159,6.076497,15.081687,6.923360,15.683162,5.810000
PL_95_obs_0,PL_95,1,PSR_J1231-1411_,2CXO J123111.3-141143,187.79710,-14.19553,9936.3,9.99,634.48130,PL,...,0.099000,4.409381,5.310000,5.420000,13.772628,5.184223,12.960586,4.469160,13.020224,4.050000
PL_97_obs_0,PL_97,1,PSR_J1135-6055_,2CXO J113508.3-605536,173.78490,-60.92683,36113.6,5.60,76.59633,PL,...,0.120000,6.256412,5.040000,5.320000,14.958607,6.324588,15.105573,6.471984,15.336959,4.550000


In [8]:
data_id = df_comb[info_col]
data_label = df_comb['class']
data_val = df_comb.drop(columns=['class']+info_col)
data_imp_norm = df_comb.copy()
for c in data_val.columns.to_list():
    data_imp_norm.loc[:,c] = (data_imp_norm.loc[:,c] - data_imp_norm.loc[:,c].mean()) / data_imp_norm.loc[:,c].std()
data_imp_norm

,src_id,num_obs,src_n,name,ra,dec,livetime,significance,likelihood,class,...,var_prob_b,photflux_aper_s,var_inter_index_h,var_inter_index_m,flux_aper_lolim_m,photflux_aper_lolim_m,flux_aper_s,photflux_aper_lolim_s,flux_aper_lolim_s,var_inter_index_s
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,33,[HPH2013]_176__________,2CXO J004245.0+411520,10.68773,41.25582,4903.5,11.31,304.46460,CV,...,0.528366,0.675600,-0.318146,-0.166969,1.309244,1.341564,0.738669,0.989158,0.914600,-0.473415
CV_1_obs_0,CV_1,104,[HPH2013]_142__________,2CXO J004242.9+411543,10.67917,41.26198,4035.0,77.08,27912.78000,CV,...,-0.607686,-1.039642,0.170627,-0.314715,-0.960304,-1.042594,-0.944054,-0.920098,-0.850090,-0.023046
CV_10_obs_0,CV_10,33,[HPH2013]_123__________,2CXO J004241.1+411544,10.67166,41.26248,4903.5,23.44,1609.49500,CV,...,0.460604,0.559184,-0.278010,-0.155546,1.196489,1.214265,0.599605,0.880598,0.841676,-0.489859
CV_11_obs_0,CV_11,33,[HPH2013]_100__________,2CXO J004237.0+411428,10.65458,41.24113,4903.5,37.03,4392.42200,CV,...,0.531663,0.606336,-0.247954,-0.147307,1.287339,1.299821,0.577984,0.948517,0.872557,-0.478435
CV_12_obs_0,CV_12,33,[HPH2013]_108__________,2CXO J004239.0+411325,10.66268,41.22376,4903.5,14.30,928.95920,CV,...,0.493328,0.541876,-0.306394,-0.166986,1.205310,1.194825,0.558321,0.889221,0.826551,-0.485704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PL_94_obs_0,PL_94,1,PSR_J1826-1256,2CXO J182608.5-125634,276.53560,-12.94294,73884.4,12.96,554.72710,PL,...,-1.027977,1.338105,-0.041046,-0.228202,0.175734,0.212133,0.947488,1.181242,1.171534,0.439613
PL_95_obs_0,PL_95,1,PSR_J1231-1411_,2CXO J123111.3-141143,187.79710,-14.19553,9936.3,9.99,634.48130,PL,...,-1.077619,-1.592019,-0.246672,-0.055176,-0.919143,-0.876846,-1.857993,-1.512051,-1.743490,-0.565670
PL_97_obs_0,PL_97,1,PSR_J1135-6055_,2CXO J113508.3-605536,173.78490,-60.92683,36113.6,5.60,76.59633,PL,...,-0.990746,0.905027,-0.409962,-0.116971,0.545555,0.514917,0.979080,0.685892,0.792557,-0.280078


In [9]:
details(data_imp_norm)

________________________________________________
Sparsity in the data : 0.00
Data shape (210, 58)
Number of sources : 
CV  	  60 	 60
LX  	  58 	 58
PL  	  92 	 92


In [10]:
verbose = 0
val_acc = []
test_acc = []
for i in tqdm(range(32)):
    src_list = pd.DataFrame()
    class_to_select = ['CV' , 'LX' , 'PL']
    df = data_imp_norm.copy()
    i_val = df[info_col]
    y_val = df['class']
    x_val = df.drop(columns=['class']+info_col)

    clf = RandomForestClassifier(
        n_estimators = 200 , n_jobs=-1
    )
    #clf = AdaBoostClassifier(base_estimator=clf ,)
    #clf = xgb.XGBRFClassifier(n_componets=200)
    #clf = xgb.XGBClassifier()
    x_train , x_test , y_train , y_test = train_test_split(x_val , y_val , test_size=0.2 , stratify=y_val, random_state=42)
    #x_train , y_train = train_value , train_label
    clf.fit(x_train , y_train)
    v_sc = clf.score(x_test , y_test)
    test_sc = clf.score(x_train, y_train)
    val_acc.append(v_sc)
    test_acc.append(test_sc)
    #if(verbose):
    print(test_sc , v_sc)

  3%|▎         | 1/32 [00:00<00:29,  1.05it/s]

0.9523809523809523 0.6428571428571429


  6%|▋         | 2/32 [00:02<00:31,  1.03s/it]

0.9523809523809523 0.6428571428571429


  9%|▉         | 3/32 [00:02<00:26,  1.09it/s]

...

0.9523809523809523 0.5952380952380952


 97%|█████████▋| 31/32 [00:25<00:00,  1.15it/s]

0.9523809523809523 0.5952380952380952


100%|██████████| 32/32 [00:26<00:00,  1.21it/s]

0.9523809523809523 0.6190476190476191


In [11]:
score = pd.DataFrame({
    "Test Score" : test_acc, 
    "Validation Score" : val_acc
})*100
score.describe().loc[['mean' , 'std' , 'min', 'max' ]]

,Test Score,Validation Score
mean,95.238095,62.202381
std,0.000000,2.475097
min,95.238095,57.142857
max,95.238095,66.666667
